* Author: Gordon Erlebacher
* Date: 2022-03-03
* Purpose: convert Sent field to either date (nb seconds from a fixed time) or an empty field.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import pandas as pd
import numpy as np
import regex as rex
import pandas as pd
from datetime import datetime
from dateutil import parser
from dateutil.tz import gettz
from unidecode import unidecode

In [ ]:
filenm = "output_0211.csv"
df = pd.read_csv(filenm)

In [ ]:
def check_multiple_names(split_comma, el, new_to_list):
    flag = False
    for com in split_comma:
        # Why isn't \s included in ^\w
        # space(s), name, space(s), name, characters, email
        if rex.match(r'\s*(\w+)\s+(\w+)[^@]+@?', com):
            flag = True
            # remove everything at the end of the first email in com
        else:
            flag = False
            break
    if flag == True:   # Lines composed just of two names separated by spaces
        # print("el: ", el)
        new_to_list[-1] = rex.sub(',', ';', el)
        #print(f"__{new_to_list[-1]}")
    return flag

In [ ]:
def check_email_or_two_words(split_comma, el, new_to_list):
    # email or two words in any order
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    mil_flag = False

    for com in split_comma:
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)(?=[^@]*$)\W*$', com):
            assign += 1
        elif rex.match(r'\s*[\w\.]+@[\w\.]+\W*$', com):
            assign += 1
    if assign == len(split_comma):
        new_to_list[-1] = rex.sub(',', ';', el)
        return True
    else:
        return False

In [ ]:
name = "  Nancy Miller "
rex.match(r'\s*(\w+)\s+(\w+)(?=[^@]*$)\W*$', name)

In [ ]:
def check_list_single_names(split_comma, el, new_to_list):   # REMOVE CELL WHEN DEBUGGED
    # Is split_comma a list of single names? 
    flag = False
    if split_comma[-1] == '' or split_comma[-1] == ' ':
        split_comma = split_comma[:-1]
    for com in split_comma:
        # Why isn't \s included in ^\w
        # space(s), name, space(s), name, characters, email
        if rex.match(r'\s*(\w+)\W*$', com):
            flag = True
            # remove everything at the end of the first email in com
        else:
            flag = False
            break
    if flag == True:   # Lines composed just of two names separated by spaces   (FLAG IS NOT TRUE!!!)
        # print("el: ", el)
        lst = []
        for i in range(0, len(split_comma), 2):
            try:
                lst.append(split_comma[i+1] + " " + split_comma[i])
            except:
                lst.append(split_comma[i])
        # print("lst: ", lst)
        #print(f"___{new_to_list[-1]}___")
        new_to_list[-1] = "; ".join(lst)
        #print(f"__{new_to_list[-1]}")
    return flag

In [ ]:
def mixture_single_and_double_names(split_comma, el, new_to_list):
    """
    # Elements are either one word or two words, ordered arbitrarily
    
    Not processed correctly:
    ------------------------
    __el:  Gary Yordon, Robert Bass, Cathy Dunklin, John Kraynak, Brian Lawrence, BobSellers, James Allen
    __el: Van Riper, Jim, Hall, Wesley, Donahoe, Judy, Tedder, Wayne
    __el: ERROR... el:   Spellman, Hella R, Gale Workman



    """
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    nb_names = []

    for com in split_comma:
        # (?=[^@]*$) means to check that there are no @ to the right
        com = rex.sub(r'AICP', '', com)
        com = rex.sub(r'KristenBurke', 'Kristen Burke', com)  # Explode
        com = rex.sub(r'WayneTedder', 'Wayne Tedder', com)  # Explode
        com = rex.sub(r'Van Riper', 'VanRiper', com)  # Explode
        com = rex.sub(r'Hella R', 'HellaR', com)  # Explode
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(2)
        elif rex.match(r'\s*([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(1)
    if assign == len(split_comma):
        # if 1 name, combine the name and the following name
        # if 2 names, keep them together
        counter = 0
        string = ''
        i = 0
        # print(f"el: __{el}__")
        # print("nb_names: ", nb_names)
        # print("len(split_comma): ", len(split_comma))
        while True:
            # print("inside while, i= ", i,)
            if nb_names[i] == 2:
                string += split_comma[i] + ';'
                i += 1
                # print("if, nb_names: 2, new i: ", i, ", string: ", string)
            elif i < len(nb_names)-1 and nb_names[i] + nb_names[i+1] == 2:
                string += split_comma[i+1] + ' ' + split_comma[i] + ';' 
                i += 2
                # print("elif, nb_names: 1, 1, new i: ", i, ", string: ", string)
            else:
                string += split_comma[i] + ';'
                break
            if i >= len(nb_names): 
                break
                
#         print(f"__el: {el}")
        new_to_list[-1] = rex.sub(';$', '', string)
        if nb_names[-2] == 2 and nb_names[-1] == 1:
            print("ERROR?: el: ", el)
            print("new_to_list[-1] = ", new_to_list[-1])
            print()
        if rex.match(r'\s*\w+\s*$', new_to_list[-1]):   # <<< ERROR
            print("ERROR... el: ", el)  
            print("new_to_list[-1] = ", new_to_list[-1])
            print()
        # print(f"   __string: {string}")
        return True
    else:
        return False

In [ ]:
name = ' Scof F. (satwell@admin.fsu.edu)'
com = rex.sub(r'\W*(\w+)\s+([A-H])\.', r'\1\2 ', name) 
print("com: ", com)

com = rex.sub(r'\W*\(([^\(]*)\)', r' \1', com)
print("com: ", com)

com = 'Mia Shargel (mshargel@gmail.com)'
com = rex.sub(r'\W*\(([^\(\)]*)\)', r' \1', com)
print("com: ", com)

com = 'Brian T.'
com = rex.sub(r'\b(\w+)\s+([A-Z])\.', r'\1\2 ', com, flags=rex.U) 
print(com)

com = "  Scott Williams  scott@hunterandharp.com "
com = unidecode(com)
com   = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\_]+@[\w\.\-]+)', com, flags=rex.U)
# com = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)', com, flags=rex.U)
print(com)


In [ ]:
name = ' DarrenM  Soto Esq.'


In [ ]:
def process_comma_field(com):
    com = rex.sub(r'AICP', '', com)
    com = rex.sub(r'KristenBurke', 'Kristen Burke', com)  # Explode
    com = rex.sub(r'WayneTedder', 'Wayne Tedder', com)  # Explode
    com = rex.sub(r'VinceLong', 'Vince Long', com)  # Explode
    com = rex.sub(r'EdMurray', 'Ed Murray', com)  # Explode
    com = rex.sub(r'Alan"Rosenzweig', 'Alan Rosenzweig', com)  # Explode
    com = rex.sub(r'MaryBarley', 'Mary Barley', com)  # Explode
    com = rex.sub('MikeVasilinda', 'Mike Vasilinda', com)
    com = rex.sub(r'Van Riper', 'VanRiper', com)  # Explode
    com = rex.sub(r'Van Pelt', 'VanPelt', com)  # Explode
    com = rex.sub(r'Hella R', 'HellaR', com)  # Explode
    com = rex.sub(r'KimRivers', 'Kim Rivers', com)  # Explode
    com = rex.sub(r'(II|III)\s*$', '', com)

    # Some spelling errors now that I have only 45 unaccounted for
    # These should be put into a function and only applied to the unnaccounted and not all the data, to 
    # minimize chance of error
    com = rex.sub(r'\- 4Genera4ons Ins4tute', '', com)
    com = rex.sub(r'Spring\.me', 'SprintTime', com)
    com = rex.sub(r'Mayor & City', 'Mayor and City', com)
    # Where did the "@" get removed? Was this in the original data? CHECK
    com = rex.sub(r'scott hunterandhar .com', 'scott@hunterandhar.com', com)
    com = rex.sub(r'Communica\.ons', 'communications', com)
    com = rex.sub(r'Jus\.n', 'Justin', com)
    com = rex.sub(r'Palmen\.eri', 'Palmentieri', com)
    com = rex.sub(r'E-mail', ' ', com)
    com = rex.sub(r'BurneDe', 'Burnette', com)  # I am almost sure.
    com = rex.sub(r'Mailto:', ' ', com, flags=rex.I)
    # Where did this misspelling come from? 
    com = rex.sub(r'Edward\.', 'Edward', com)
    # special cases where first and last names are separated by dots, which must be replaced by a space
    com = rex.sub(r'Tom\.Coe', 'Tom Coe', com)
    com = rex.sub(r'John\.Marks', 'John Marks', com, flags=rex.I)
    com = rex.sub(r'Andrew\.Gillum', 'Andrew Gillum', com, flags=rex.I)
    com = rex.sub(r'Gil\.Ziffer', 'Gil Ziffer', com, flags=rex.I)
    com = rex.sub(r'\bAr\.e\b', 'Artie', com, flags=rex.I)
    com = rex.sub(r'\-?\s?E\.D\.I\.F\.Y\. Mentoring', '', com, flags=rex.I)
    # Joey did not capture the word Group on occasion, when it was isolated to the following line
    com = rex.sub(r'ISS\/TI (Server Group|Server)', 'ISSTI ServerGroup', com, flags=rex.I)

    # Found in the To: list  (It was originall "Mike (Sheriff's Office) Wood"
    com = rex.sub('Mike  Sheriffs Office  Wood', 'Mike Wood', com, flags=rex.I)
    com = rex.sub(r'Anita.Favors', 'Anita Favors', com)
    com = rex.sub(r'\- Tallahassee Downtown Improvement Authority', '', com)
    com = rex.sub(r'Wilma\. Sims@talgov.com', 'Wilma.Sims@talgov.com', com)
    com = rex.sub(r'(Cur\.s|Cur\-s)', 'Curtis', com)
    com = rex.sub(r'Sa-shrp_99@yahoo', 'Satishrp_99@yahoo', com, flags=rex.I)  
    com = rex.sub(r'^:', '', com)
    com = rex.sub(r'ac\.on', 'action', com, flags=rex.I)
    com = rex.sub(r'Tallahassee-Leon County Status of Girls and Women', 'TallahasseeLeonCountyStatus Girls Women', com)
    com = rex.sub(r'ScoY', 'Scott', com, flags=rex.I)  # \b does not work
    com = rex.sub(r'Levi\+cus', 'Leviticus', com)
    com = rex.sub(r'Parks \& Rec - Senior Mgmt. Team', 'Parksand Rec', com)
    com = rex.sub(r'Pe\^s', r'Pettis', com)
    com = rex.sub(r'@%0ctlh.org', r'@fbctlh.org', com)
    # print('bef com: ', com)
    com = rex.sub(r'Asst\. Chief Glenn Sapp', 'Glenn Sapp', com)
    # print('aft com: ', com)
    com = rex.sub(r'Col. Ronald Joe', 'Ronald Joe', com)
    com = rex.sub(r'Apopka \-', '', com)
    com = rex.sub(r'Connie Diaz - ', 'Connie Diaz ', com)
    com = rex.sub(r'Greenacres Aide: ', '', com)
    com = rex.sub(r'Port St\. Lucie', 'Port StLucie', com)
    com = rex.sub(r'Krebs Assistant:', '', com)
    com = rex.sub(r'\-\- Baugh.*?view', '', com)
    com = rex.sub(r'Mustian Aide:', '', com)
    com = rex.sub(r'P\. C\. Wu\:? ', 'PC Wu', com) 
    com = rex.sub('PC\s?Wu Asst:', '', com)
    com = rex.sub(r'Starkey Aide \-\-', '', com)
    com = rex.sub(r'Esq\.', '', com)
    com = rex.sub(r'Rep\. ', ' ', com)
    com = rex.sub(r'Pi\*nger', 'Pittinger', com)
    com = rex.sub('bagge\%0@gtlaw.com', 'baggettf@gtlaw.com', com)
    com = rex.sub('chriskra\*@kra\*nissan.com', 'chriskraft@kraftnissan.com', com)
    com = rex.sub('Scof', 'Scott', com)
    com = rex.sub('P\&R', '', com)
    com = rex.sub('UWBB \-', '', com)
    # Two people in on slot of the header. Therefore, create a new person: BergerSingerman MelanieHines
    com = rex.sub(r'BergerSingerman \- Melanie Hines', 'BergerSingerman MelanieHines', com)
    com = rex.sub(r'RickOppenheim', 'Rick Oppenheim', com)
    com = rex.sub(r'Chuck \& Pa', 'ChuckandPattie', com)
    com = rex.sub(r'Dario4s', 'Dariotis', com)
    com = rex.sub(r'\-  S \& D', '', com)
    # print("before, com: ", com)
    com = rex.sub(r'Parks \& Rec - Senior Mgmt\. Team', 'ParksandRec SeniorMgmtTeam', com)
    com = rex.sub(r'Parks\&Rec\-', '', com)
    # print("after, com: ", com)
    # \S means "everything except \s"
    com = rex.sub('IMCEAE\S+', '', com) 
    com = rex.sub('\- TLH', '', com)
    com = rex.sub(r'Steve & Sue Ecenia', 'SteveandSue Ecenia', com)
    com = rex.sub(r'Vicky & Steve', 'VickyandSteve', com)
    com = rex.sub(r'Dana & Gene', 'DanaandGene', com)
    com = rex.sub(r'Brenda & Wayne', 'BrendaandWayne', com)
    com = rex.sub(r'MACKENZIE\.ANNE', 'Anne MacKenzie', com)
    com = rex.sub(r'ThomasK\.', 'ThomasK', com)
    com = rex.sub(r'Chris4cA\.', 'ChristicA', com)
    com = rex.sub(r'\!', '', com)
    com = rex.sub(r'KarenRubin \- TAPP', 'Karen Rubin', com)
    com = rex.sub(r'Chris\.ansen', 'Christiansen', com)
    com = rex.sub('JamiColeman', 'Jami Coleman', com)
    # Name found manually in output.csv file and confirmed on internet
    com = rex.sub('harpcomm \.', 'Jayme Harpring harpcomm@gmail.com', com)
    # Office of Tallahassee Sheriff, confirmed on internet
    com = rex.sub('A\.J\.Smith', 'AJ Smith', com)
    com = rex.sub('Van Pelt', 'VanPelt', com)
    com = rex.sub('Dus\.n', 'Dustin', com)
    com = rex.sub('Aus\.n', 'Austin', com)
    com = rex.sub('City Managers Conf\. Rm\. 4B', 'City ManagersRm4B', com)
    com = rex.sub(r'William \\Bill\\ Tucker', 'William Bill Tucker', com)
    com = rex.sub(r'Whi\*ield', 'Whitfield', com)
    com = rex.sub(r'Jonathan.Kil atrick tal ov.com', 'JOnathan.Kilpatrck@talgov.com', com)
    com = rex.sub(r'Lalu\?en', 'Lauren', com)
    com = rex.sub(r'Mr\. Evans Ms\. Parades', 'MrEvans MsParades', com)
    com = rex.sub(r'Chris\.an', 'Christian', com)
    com = rex.sub(r'Rob Davis, P\.E\.', 'Rob Davis', com)
    com = rex.sub('Culpeppcr Construction Company,lnc.', 'Culpepper Construction', com)
    com = rex.sub('Chris4ne', 'Christine', com)
    com = rex.sub('MaYhew', 'Matthew', com)
    com = rex.sub('Brown Jon D.', 'JonD Brown', com)
    com = rex.sub(r'\- Tallahassee Downtown Improvement Authority', '', com)
    com = rex.sub(r'sixelconsul4ng.com', 'sixelconsulting.com', com)

    # if three strings, combine the first two (assumes first, middle and last name)
    # print("     ...bef: ", com)
    ## Cheap solution (need better one) to reduce 3 or 4 words down to 2 (no consideration of emails)
    # Make sure the three words do not have emails
    # (?!\w*@) checks that the word following the 2nd word has no '@'
    com = rex.sub(r'(\w+)\s+(\w+)\s+(?!\w*@)(\w+)', r'\1\2 \3', com, flags=rex.U)
    com = rex.sub(r'(\w+)\s+(\w+)\s+(?!\w*@)(\w+)', r'\1\2 \3', com, flags=rex.U)
    com = rex.sub(r'(\w+)\s+(\w+)\s+(?!\w*@)(\w+)', r'\1\2 \3', com, flags=rex.U)
    # print("     ...aft: ", com)

    com = rex.sub(r'\b([\w\-]+)\s+([A-Z])\.', r'\1\2 ', com) 
    # Change (xxxx) to xxxxx where xxxx is anything without parenthetis
    com = rex.sub(r'\W*\(([^\(\)]*)\)', r' \1', com)

    com = rex.sub(r'(Jr\.|Dr\.|Rev\.| Sr\.)', r' ', com)

        
    return com
    

In [ ]:
def mixture_email_single_and_double_names(split_comma, el, new_to_list, debug):
    """
    # Elements are either (one word or email) or two words, ordered arbitrarily
    
    Not processed correctly:
    ------------------------
    __el:  Gary Yordon, Robert Bass, Cathy Dunklin, John Kraynak, Brian Lawrence, BobSellers, James Allen
    __el: Van Riper, Jim, Hall, Wesley, Donahoe, Judy, Tedder, Wayne
    __el: ERROR... el:   Spellman, Hella R, Gale Workman
    ___el: ERROR... el:   Faris, Alison McCray, Patricia Edwards, Ashley, Bowens, Spencer, Barber, Cynthia, Goad, Reese, Lavin, Raoul, paige.tallahasseedowntown@gmail.com, Gaines, Jerome
    RROR?: el:   Faulkenberry, Gary Yordon, brian.cook@HCAHealthcare.com, dale Neeley, Jeneane Dunkle, Tim
    ERROR?: el:   andrew.gillum@talgov.com, rick.fernandez@talgov.com, scof, Nancy Miller, gil.ziﬀer@talgov.com,  (WHAT is "scof"?)
RROR?: el:   Adrianne Miller, Aubry Hawks, Anneliese  , Gary Yordon, Cynthia.Barber@talgov.com
ERROR... el:   Faulkenberry, Gary Yordon, brian.cook@HCAHealthcare.com, dale Neeley, Jeneane Dunkle, Tim
ERROR?: el:   jackpofer@me.com, dougwray@comcast.net, Lauren Wray, adam, Susan Moore, Judi Pofer, Chuck

 SBELLEME@broward.org, chris.marlin@lennar.com, SPa†son@rsilink.com, lauren@laurenbook.com, mmk@portbiz.org, 
 scernak@broward.org, Chrisene.Wydra@lennar.com, claire@laurenbook.com, afazzano@broward.edu, tbain@miamigardens-ﬂ.gov, 
 sdick@talchamber.com, Pingree, Ben, Kerri.Post@VisitTallahassee.com, Fernandez, Rick, Curry, Chris, Maddox, Scof, 
 paige.tallahasseedowntown@gmail.com, agonzalez@visi‰lorida.org, ron@sachsmedia.com, rgreen2505@me.com, 
 victoria@sachsmedia.com, lehirst@fsu.edu, scof@scofmaddox.com, Atwell, Scof F. (satwell@admin.fsu.edu), Bryant, 
 
 Scof F. (satwell@admin.fsu.edu)
 
 Aundra, Elyea, Janice, carmella@sachsmedia.com, Gregory Haile
 
 Wilcox, Barry, Wiebler, Brian T., Doherty, Megan, Bryant, Cherie
 
 Sampson-Jones, Lavonia <Lavonia.Sampson-Jones@talgov.com>, Johansen, Jay <Jay.Johansen@talgov.com
 
 Dr. Audra Pi_man, Straughn, Rachael, Paige Carter-Smith, Poole, Carrie
    """
    # print("enter mixture")
    # print("el: ", el)
    # print("split_comma: ", split_comma)
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    nb_names = []
    emails = []
    scotflag = False
    DEBUG = debug
    # DEBUG = True
    
    if DEBUG: 
        print(el)

    for com in split_comma:
        if DEBUG: 
            print(f"==> com: __{com}__")
            
        # Processed already prior to calling this function
        #com = process_comma_field(com)   # <<< NEW for reusability

        if DEBUG:
            print(f"   end com: __{com}__")
 
        emails.append(False)
        email = None
        first_name = None
        last_name = None
        
        # 2 names
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(2)
        # 2 names and email
        elif rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U):
            match = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U)
            assign += 1
            nb_names.append(2)
            emails[-1] = True
            first_name = match.group(1)
            last_name = match.group(2)
            email = match.group(3)
        # 1 name 
        elif rex.match(r'\s*([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(1)
        # single email
        elif rex.match(r'\W*[\w+\._\-]+@[\w+\.\-]+', com, flags=rex.U):
            assign += 1
            nb_names.append(1)
            emails[-1] = True
        # single name + email
        elif rex.match(r'\s*([\w\-]+)\s\W*([\w+\.\-]+@[\w\.\-]+)', com, flags=rex.U):
            # print("...match")
            match = rex.match(r'\s*([\w\-]+)\s\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U);
            # print("match: ", match)
            #print("... com: ", com, ",     match: ", match.groups())
            assign += 1
            nb_names.append(1)
            emails[-1] = True
            first_name = match.group(1)
            email = match.group(2)
            # print("name: ", name, ",   email: ", email)
        else:
            if DEBUG:
                print("NO MATCH, com: ", com)

    # print("assign:  ", assign, ",    len(split_comma)= ", len(split_comma))      
    if assign == len(split_comma):
        # if 1 name, combine the name and the following name
        # if 2 names, keep them together
        counter = 0
        string = ''
        i = 0
        # print(f"el: __{el}__")
        # print("nb_names: ", nb_names)
        # print("len(split_comma): ", len(split_comma))
        while True:
            # print("inside while, i= ", i,)
            if nb_names[i] == 2:
                string += split_comma[i] + ';'
                i += 1
                # print("if, nb_names: 2, new i: ", i, ", string: ", string)
            elif emails[i] == True and not first_name:
                string += split_comma[i] + ';'
                i += 1
            elif i < len(nb_names)-1 and nb_names[i] + nb_names[i+1] == 2:
                if first_name:
                    string += first_name + ' ' + split_comma[i] + ' ' + email + ';' 
                else:
                    string += split_comma[i+1] + ' ' + split_comma[i] + ';' 
                i += 2
                # print("elif, nb_names: 1, 1, new i: ", i, ", string: ", string)
            else:
                string += split_comma[i] + ';'
                break
            if i >= len(nb_names): 
                break
                
        # Remove last semi-colon if end of string
        new_to_list[-1] = rex.sub(';\s*$', '', string)
        # print("new_to_list[-1]= ", new_to_list[-1])
        # if nb_names[-2] == 2 and nb_names[-1] == 1:
        #     print("ERROR?: el: ", el)
        #     print("new_to_list[-1] = ", new_to_list[-1])
        #     print()
        # if rex.match(r'\s*\w+\s*$', new_to_list[-1]):   # <<< ERROR
        #     print("ERROR... el: ", el)  
        #     print("new_to_list[-1] = ", new_to_list[-1])
        #     print()
        # print(f"   __string: {string}")
        return True
    else:
        return False

In [ ]:
name = " Dr. "
rex.sub(".*\bDr.", '', name)

name = "This island is beautiful"
rex.sub("\bis\b", "gordon", name)


el = 'ads ( gord (adf.;afdlkj com ) asdf'
el = "as (adf (adf ) gas)" # should return "as", but returns "as gas)"
match = rex.match(r'.*\([^@\(\)]*@[^@\)]*\)', el)
if  match:  # there is an email, so remove parenthes
    el = rex.sub(r'\(([^\(\)]*)\)', r'\1', el)
else: 
    # Remove everything within parenthesis along with the parenthesis (knowing that email is excluded)
    el = rex.sub(r'\([^\)]*\)', '', el)
print(el)

In [ ]:
df.columns

In [ ]:
to_list = df['CC'].values   # 170 not processed out of 71,000
# to_lst = df['To'].values  # 566 not processed out of 71,000

not_processed = []
new_to_list = []
header_with_date = []  # should not happen. Email is probalby incorrect
debug = False

if debug:
    to_list = [ 
    ]
if to_list == []:
    debug = False


for el1 in to_list[0:]:
    # # print("el1: ", el1)
    new_to_list.append(el1)
    # print("new_to_list: ", new_to_list)
    # print("el1: ", el1)
    # print("----------------")
    # # print(el1, new_to_list[-1])
    # continue   # TEMPORARY

    if type(el1) != str:
        #not_processed.append(el1)
        #print("el1= ", el1, ",  INVALID")
        new_to_list[-1] = 'INVALID_TYPE' # of NaN
        continue

    el = unidecode(el1)
    # remove  single and double quotes
    el = rex.sub(r'[\'\"]', '', el)
    # transform abbreviations of the form A. by A
    #el = rex.sub(r' (\w)\.', r' \1 ', el)  # adding this increases non-processed to 436 from 397
    # Remove (\w+)
    #el = rex.sub(r'\([\w\-\s]+\)', ' ', el)
    
    # remove parenthesis, square and angular brackets
    el = rex.sub(r'[\{\}\(\)\<\>\[\]]', ' ', el)
    # remove parenthesis around emails
    # remove parenthesis, square and angular bracket and their content, IF there is no mail symbol inside
    #el = rex.sub(r'\([^\)@]*\)', '', el)  # also removes emails within brackets
    
    # if email, remove brackets, else leave brakcket
    # Note: sometimes emails my be screwed up with spaces
    # I cannot handle parenthesis within parenthesis correctly
    #  The following does not work: el = "as (adf (adf ) gas)"
    match = rex.match(r'.*\([^@\(\)]*@[^@\)]*\)', el)
    if  match:  # there is an email, so remove parenthes
        el = rex.sub(r'\(([^\(\)]*)\)', r'\1', el)
    else: 
        # Remove everything within parenthesis along with the parenthesis (knowing that email is excluded)
        el = rex.sub(r'\([^\)]*\)', '', el)

    
    # Special case that cannot easily be handle in another way
    el = rex.sub(r'Powell, Jr., Rudy', 'Rudy Powell', el)
    el = rex.sub(r'Hightower, Jr., Cecil', 'Cecil Hightower', el)
    el = rex.sub(r', Jr\.', '', el)
    el = rex.sub(r'(,\s*,|,\s*III\s*,|,\s*Jr\.\s*,)', r',', el)  # I DO NOT KNOW WHETHER THIS WILL ALWAYS WORK
    el = rex.sub(r'\sAICP\s*$', '', el)
    el = rex.sub(r'\?Tedder, Wayne \?McCraw, Rick', 'Wayne Tedder, Rick McCraw', el)
    el = rex.sub(r'The Village Square', '', el)
    el = rex.sub(r',\s*(P\.E\.|P\.A\.|lnc\.|Esq\.|Inc\.|P\.L\.[^,]*Brunch)', '', el)
    el = rex.sub(r', ,', ',', el)
    
    # Should be in a separate module for reusability
    el = rex.sub(r'\- Tallahassee Downtown Improvement Authority', '', el)
    if debug:
        print("el: ", el)
    match_month = rex.match(r'.*\b(January|February|March|April|May|June|July|August|September|October|November|December)\b', el, flags=rex.I)
    match_number = rex.match('.*\s\d+\s', el)
    new_to_list[-1] = el

    #ignore_dates_stopwords(el)
    # Make sure that the match does not succeed because somebody's name is May
    if match_month and match_number:
        header_with_date.append(el)
        not_processed.append(('DATE: ', el))
        new_to_list[-1] = 'INVALID_DATE'
        continue
    if rex.match(r'.* (that|the) ', el, flags=rex.I):
        new_to_list[-1] = 'INVALID_STOP_WORD'
        not_processed.append(('STOP WORD: ', el))
        continue

    try:
        splits = el.strip().split(";")
    except:
        new_to_list[-1] = 'INVALID_EXCEPTION'
        not_processed.append(('EXCEPTION: ', el))
        continue
    sz = len(splits)
    
    if sz != 1:
        # Process all the fields
        for i, com in enumerate(splits):
            splits[i] = process_comma_field(com)
        # Recreate el
        el = ';'.join(splits)

        if debug:
            print("continue, sz: ", sz)
        continue
        
    # Split with comma   
    # last, first gor@email, last, first fran@gmail

    split_comma = el.strip().split(',')
    for i, com in enumerate(split_comma): 
        split_comma[i] = process_comma_field(com)
    el = ','.join(split_comma)
    
    split_at = el.split('@')
    # if debug: print("split_comma, split_at: ", len(split_comma), len(split_at))
    
    if len(split_comma) == 1:
        new_to_list[-1] = el
        continue
        
    # if len(split_comma) <= 2:
    #     if debug: print("split_comma <= 2")
    #     continue 
        
    if len(split_comma) == len(split_at) or len(split_comma) == (len(split_at) - 1):
        # simply replace commas by semi-colons
        new_to_list[-1] = rex.sub(',', ';', el)
        # if debug: 
        #     print("len(split_comma) == len(split_at) ...")
        continue

    # The com should be processed once before calling these different routines, with regex.sub 
    # TODO


    # each split by comma should be EITHER two words or have an email
    if debug:
        print("before check_multiple_names")
    flag = check_multiple_names(split_comma, el, new_to_list)
    if flag: continue

    if debug:
        print("before check_list_single_names")
    flag = check_list_single_names(split_comma, el, new_to_list)
    if flag: continue

    # process elements.
    # - If full name (2 words), assign 1. 
    # - if just an email (1 string), assign 1
    # if the string is composed of just 1's, replace commas by semi-colons

    if debug:
        print("before check_email_or_two_words")
    flag = check_email_or_two_words(split_comma, el, new_to_list)
    if flag: continue

    if debug:
        print("before mixture_email_single_and_double_names")
    # flag = mixture_single_and_double_names(split_comma, el, new_to_list)
    flag = mixture_email_single_and_double_names(split_comma, el, new_to_list, debug=debug)
    if flag: continue

    new_to_list[-1] = 'INVALID_UNKNOWN'
    not_processed.append(('UNKNOWN CAUSE', el))

    # for com in split_comma:
    #     print(f"__{com}__")
    # print(len(split_comma), len(split_at), el)

    # I would also lines where each element is either: 
    #  two or more names separated by spaces followed by a string of characters including the @ with no spaces
    #   (which is more general)
        
print(len(not_processed), len(new_to_list), len(to_list))



In [ ]:
count = 0
print(len(not_processed))
for el in not_processed:
    if rex.match(r'.*UNKNOWN', el[0]):
    # if True:
        print(f"==> {el}")
        count += 1


In [ ]:
count, len(new_to_list), len(to_list)

In [ ]:
new_to_list[0], '_____', to_list[0]

In [ ]:
# the number 4 often, but not always, represents 'ti' because of improper translation from pdf I assume. 
# Exception: ...@awards4u.com  . It is obvious that the email is not @awardstiu. 
# I do not handle these exceptions. For the purposes of analysis, it really does not matter if all the emails with 4 are translated
# Another exception: amy4hand@gmail.com, which is correct I believe
# for i, el in enumerate(zip(to_list[0:], new_to_list[0:])):
#     if rex.match('.*([a-zA-Z]4[a-zA-Z])', el[1]):
#         new_to_list[i] = rex.sub(r'([a-zA-Z])4([a-zA-Z])', r'\1ti\2', new_to_list[i])
#         print("==> ", el[0])
#         print("   --> ", new_to_list[i])

In [ ]:
for el in zip(to_list[0:1000], new_to_list[0:1000]):
    print("==> ", el[0])
    print("  --> ", el[1])
    
#   -->  Sean Marston  smarston@ucceng.com , Hays Layerd  hlayerd@think3d.net


In [ ]:
name = '  @ asf'
rex.match(r'[^\w]+@', name)

In [ ]:
name = '2/16/17 3:25 PM (GMT-05:00)'
name = rex.sub(r'(\d?\/\d)\/(\d\d )', r'\1/20\2', name)
rex.match(r'(\d+?\/\d+?)\/(\d\d )', name)

In [ ]:
exceptions = []
date_dict = []
time_zone = []
for sent in sent_lst:
    normalize(sent, exceptions, date_dict, time_zone)
    
len(exceptions), len(sent_lst), df.shape

In [ ]:
exceptions

* 'Friday, May 13, 201611:18 AM': datetime.datetime(2022, 5, 13, 18, 16, 11),
* 

In [ ]:
dates_orig = []
dates_new = []
dates_date = []
date_adj = []
timestamp = []
for i, (tz_el, el) in enumerate(zip(time_zone, date_dict)):
    dates_orig.append(el[0])
    dates_new.append(el[1])
    dates_date.append(el[2])
    try:
        # subtract 5 hours to convert back to Tallahassee time
 
        if False:  # take daylight savings properly into account
            if tz_el[1] == 'S':  # adjust time if Savings
                timestp -= 3600
        else:  
            timestp = el[2].timestamp() - 5 * 3600 
            
        timestamp.append(timestp)
        # Transform time from timestamp back to UTC
        dtime = datelib.timestampToDateTimeUTC(timestp)
        date_adj.append(dtime)
    except:
        timestamp.append(-1)
        date_adj.append(('',''))
    
df1 = pd.DataFrame({'orig':dates_orig, 'new':dates_new, 'date':dates_date, 'TZ': time_zone, 'date_adj': date_adj, 'timestamp': timestamp})
print(len(date_dict), len(dates_orig), df.shape)

df1.to_csv("dates.csv", index=0)

In [ ]:
df.shape, df1.shape

In [ ]:
# Add new columns: new date and time, and number of seconds since 1970
# timestamp: seconds since 1970
# dates_orig: original send column
# date_adj[0]: adjusted date
# date_adj[1]: adjusted time
adj_date = []
adj_time = []
for dat_tim in date_adj:
    adj_date.append(dat_tim[0])
    adj_time.append(dat_tim[1])
    
df['timestamp'] = timestamp
df['date_sent'] = adj_date
df['time_sent'] = adj_time
df.columns

df.to_csv("output_stats_timestamps.csv.gz", index=0)

In [ ]:
timestamp;
df.columns